In [1]:
import torch
from torch import nn  
import matplotlib.pyplot as plt
import pandas as pd 
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
import torch.utils.data as data_utils


In [3]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.layer1 = nn.Linear(in_features=23,out_features=256)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(256, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 4)
            
        )

    def forward(self, x):
        x = self.layer1(x)
        logits = self.linear_relu_stack(x)
        return logits
    

NeuralNetwork(
  (layer1): Linear(in_features=23, out_features=256, bias=True)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=256, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=4, bias=True)
  )
)


In [5]:
model = NeuralNetwork().to("cuda")
model.load_state_dict(torch.load("modelo99"))

<All keys matched successfully>

In [6]:
df = pd.read_csv("minMaxScaledTEST.csv",index_col=0)

In [7]:
df.shape

(296786, 53)

In [9]:
columns = list(df.columns)
output_columns = ["RENDIMIENTO_GLOBAL"]
input_columns = list(filter(lambda x: not 'RENDIMIENTO_GLOBAL' in x, columns))
input_columns2 = list(filter(lambda x: not 'ESTU_PRGM_DEPARTAMENTO' in x, columns))

In [17]:
input_columns2

['ESTU_VALORMATRICULAUNIVERSIDAD',
 'ESTU_HORASSEMANATRABAJA',
 'FAMI_ESTRATOVIVIENDA',
 'FAMI_TIENEINTERNET',
 'FAMI_EDUCACIONPADRE',
 'FAMI_TIENELAVADORA',
 'FAMI_TIENEAUTOMOVIL',
 'ESTU_PRIVADO_LIBERTAD',
 'ESTU_PAGOMATRICULAPROPIO',
 'FAMI_TIENECOMPUTADOR',
 'FAMI_EDUCACIONMADRE',
 'ESTU_PRGM_ACADEMICO_Administración y Negocios',
 'ESTU_PRGM_ACADEMICO_Ciencias Exactas',
 'ESTU_PRGM_ACADEMICO_Ciencias de la Salud',
 'ESTU_PRGM_ACADEMICO_Comunicación y Periodismo',
 'ESTU_PRGM_ACADEMICO_Deporte y Actividad Física',
 'ESTU_PRGM_ACADEMICO_Derecho y Arquitectura',
 'ESTU_PRGM_ACADEMICO_Gastronomía',
 'ESTU_PRGM_ACADEMICO_Ingeniería',
 'ESTU_PRGM_ACADEMICO_Licenciaturas y Educación',
 'ESTU_PRGM_ACADEMICO_Otro',
 'ESTU_PRGM_ACADEMICO_Tecnología de la Información']

In [13]:
df[input_columns2]

,ESTU_VALORMATRICULAUNIVERSIDAD,ESTU_HORASSEMANATRABAJA,FAMI_ESTRATOVIVIENDA,FAMI_TIENEINTERNET,FAMI_EDUCACIONPADRE,FAMI_TIENELAVADORA,FAMI_TIENEAUTOMOVIL,ESTU_PRIVADO_LIBERTAD,ESTU_PAGOMATRICULAPROPIO,FAMI_TIENECOMPUTADOR,...,ESTU_PRGM_ACADEMICO_Ciencias Exactas,ESTU_PRGM_ACADEMICO_Ciencias de la Salud,ESTU_PRGM_ACADEMICO_Comunicación y Periodismo,ESTU_PRGM_ACADEMICO_Deporte y Actividad Física,ESTU_PRGM_ACADEMICO_Derecho y Arquitectura,ESTU_PRGM_ACADEMICO_Gastronomía,ESTU_PRGM_ACADEMICO_Ingeniería,ESTU_PRGM_ACADEMICO_Licenciaturas y Educación,ESTU_PRGM_ACADEMICO_Otro,ESTU_PRGM_ACADEMICO_Tecnología de la Información
0,0.142857,0.25,0.500000,0.0,0.727273,0.0,1.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
1,0.571429,0.75,0.333333,0.0,0.545455,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.428571,0.00,0.500000,0.0,0.454545,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,0.428571,0.75,0.166667,1.0,0.272727,0.0,1.0,1.0,1.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.571429,0.50,0.333333,0.0,0.545455,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
296781,0.571429,1.00,0.166667,0.0,0.272727,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
296782,0.428571,0.00,0.666667,0.0,0.909091,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
296783,0.571429,1.00,0.500000,0.0,0.545455,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
296784,0.857143,0.50,0.500000,0.0,0.909091,0.0,1.0,1.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [11]:
X = torch.from_numpy(df[input_columns2].to_numpy()).type(torch.float).to(device)

In [15]:
X

tensor([[0.1429, 0.2500, 0.5000,  ..., 0.0000, 1.0000, 0.0000],
        [0.5714, 0.7500, 0.3333,  ..., 0.0000, 0.0000, 0.0000],
        [0.4286, 0.0000, 0.5000,  ..., 0.0000, 0.0000, 0.0000],
        ...,
        [0.5714, 1.0000, 0.5000,  ..., 0.0000, 0.0000, 0.0000],
        [0.8571, 0.5000, 0.5000,  ..., 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.5000,  ..., 0.0000, 0.0000, 0.0000]],
       device='cuda:0')

In [12]:
with torch.no_grad():
    pred = model(X)



RuntimeError: mat1 and mat2 shapes cannot be multiplied (296786x22 and 23x256)